[View in Colaboratory](https://colab.research.google.com/github/akhil927/Deep-Learning/blob/master/Lyrics_generation.ipynb)

In [0]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from google.colab import files

In [2]:
# files.upload()
files.os.listdir()

['datalab',
 'textfile.txt',
 '.keras',
 '.nv',
 '.forever',
 'textfile (1).txt',
 '.cache',
 '.config',
 '.ipython',
 '.local']

In [0]:
with open('textfile.txt') as f:
    data = f.read()
data = data.split()
words = list(set(data))
classes = len(words)

In [0]:
dictionary = {}
reverse_dictionary = {}
for i, word in enumerate(words):
    dictionary[word] = i
    reverse_dictionary[i] = word
    
for i, each in enumerate(data):
    data[i] = dictionary[each]

In [0]:
X = []
Y = []
for i in range(len(data)-3):
    x = data[i:i+3]
    y = data[i+3]
#     print(x, y)
    output = np.zeros([classes])
    output[y] = 1
    X.append(x)
    Y.append(output)
    
X = np.asarray(X)
X = X.reshape(X.shape[0], X.shape[1], 1)
Y = np.asarray(Y)

In [6]:
print(X.shape, Y.shape)

(716, 3, 1) (716, 328)


In [0]:
trainX = tf.placeholder(tf.float32, [None, 3, 1])
trainY = tf.placeholder(tf.float32, [None, classes])

In [0]:
hidden_units = 128
cell = tf.nn.rnn_cell.LSTMCell(hidden_units, state_is_tuple=True)
val, state = tf.nn.dynamic_rnn(cell, trainX, dtype=tf.float32)
val = tf.transpose(val, [1,0,2])
last = tf.gather(val, int(val.get_shape()[0])-1)

In [0]:
weight = tf.Variable(tf.truncated_normal([hidden_units, int(trainY.get_shape()[1])]))
bias = tf.Variable(tf.constant(0.1, shape=[trainY.get_shape()[1]]))

prediction = tf.matmul(last, weight) + bias

In [10]:
cost = tf.losses.softmax_cross_entropy(trainY, prediction)
optimizer = tf.train.AdamOptimizer().minimize(cost)
mistakes = tf.not_equal(tf.arg_max(trainY,1), tf.arg_max(prediction,1))
error = tf.reduce_mean(tf.cast(mistakes, tf.float32))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Instructions for updating:
Use `argmax` instead


In [0]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [0]:
batch_size = 40
no_of_batches = int(len(X) / batch_size)
epochs = 10000
for i in range(epochs):
    ptr = 0
    for j in range(no_of_batches):
        inputX, inputY = X[ptr: ptr+batch_size], Y[ptr:ptr+batch_size]
        ptr += batch_size
        sess.run(optimizer, feed_dict={trainX: inputX, trainY: inputY})
    if i % 100 == 0:
        print("Epoch - ", i, " Completed!")


In [0]:
incorrect = sess.run(error, {trainX: X, trainY: Y})
pred = sess.run(prediction, {trainX: X})

In [50]:
while True:
    sentence = input()
    sentence = sentence.strip()
    tokens = sentence.split()
    testX = [dictionary[tokens[i]] for i in range(len(tokens))]
    testX = np.array(testX)
    testX = np.reshape(testX, (-1, testX.shape[0], 1))
    for i in range(100):
        pred = sess.run(prediction, {trainX: testX})
        sentence += ' ' + reverse_dictionary[np.argmax(pred)]
        testX[:,0:2,:] = testX[:,1:,:]
        testX[0,2,0] = np.argmax(pred)
    print(sentence)

The truth and
The truth and my lies now are falling like the rain So let the river run My name's (ooh), my name's (ooh) River (ooh), we'll let the river run My name's (ooh), my name's (ooh) River (ooh), we'll let the river run My name's (ooh), my name's (ooh) River (ooh), we'll let the river run My name's (ooh), my name's (ooh) River (ooh), we'll let the river run My name's (ooh), my name's (ooh) River (ooh), we'll let the river run My name's (ooh), my name's (ooh) River (ooh), we'll let the river run My name's (ooh), my name's (ooh) River (ooh), we'll
Always the bridesmaid,
Always the bridesmaid, never "The bride, hey!" Fuck can I say? If life was a highway And deceit was an enclave, I'd be swerving in five lanes Speeds at a high rate, like I'm slidin' on ice, maybe That's why I may have came at you sideways I can't keep my lies straight But I made you terminate my baby This love triangle left us in a wreck, tangled What else can I say? If life was a highway And deceit was an enclave,

KeyboardInterrupt: ignored